In [1]:
import pandas as pd
import numpy as np
import warnings

data = pd.read_csv('../data/ERP_data.csv')
labels = list(data.columns.values)
del labels[0]
del labels[0]

targets = data['Phenotype']
del data['Subject']
del data['Phenotype']

In [2]:
from sklearn import preprocessing, feature_selection, cross_validation, ensemble

folds = 10

imp = preprocessing.Imputer()
data = imp.fit_transform(data, targets)
data = preprocessing.scale(data)
'''
anova_selection = feature_selection.SelectKBest(feature_selection.f_classif, k=12)
features_selected = anova_selection.fit(data, targets).get_support(indices=True)
labels = pd.Series(labels)[features_selected]
print(labels)
data = anova_selection.transform(data)
'''
data = pd.DataFrame(data, columns=labels)

clf = ensemble.GradientBoostingClassifier(min_samples_split=30, learning_rate=0.5, n_estimators=110)

'''
features_selected = []
for i in range(50):
    cross_val = cross_validation.StratifiedKFold(targets, n_folds=folds, shuffle=True)
    rfecv = feature_selection.RFECV(clf, cv=cross_val)
    rfecv.fit(data, targets)
    features_selected.append(rfecv.get_support())
    
feature_probs = np.mean(np.asarray(features_selected).astype(float), axis=0)
print(feature_probs)
features_selected = (feature_probs >= 0.5).nonzero()[0]
labels = pd.Series(np.asarray(labels))[features_selected]
print(labels)
data = data[labels]
'''

data = data[['P3aDACz', 'P200DAF3', 'P200DAF4', 'P3aDAvgACz',
             'P3aDAvgAF3', 'P3aDAP4', 'P3aDAvgAFz', 'P200DAvgAF4', 'P3aDAvgAF4']]

In [3]:
from sklearn import metrics

accuracies = []
precisions = []
recalls = []
fscores = []
confusions = []
warnings.filterwarnings("ignore")
for i in range(200):
    cross_val = cross_validation.StratifiedKFold(targets, n_folds=folds, shuffle=True)
    preds = cross_validation.cross_val_predict(clf, data, targets, cv=cross_val)
    accuracy = metrics.accuracy_score(targets, preds)
    precision, recall, fscore, support = metrics.precision_recall_fscore_support(
        targets, preds, average='binary', pos_label='AD')
    confusion = metrics.confusion_matrix(targets, preds, labels=['HC', 'AD'])
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    fscores.append(fscore)
    confusions.append(confusion)

print("Accuracy: {0}, with std: {1}".format(np.mean(accuracies), np.std(accuracies)))
print("Precision: {0}".format(np.mean(precisions)))
print("Recall: {0}".format(np.mean(recalls)))
print("F1 Score: {0}".format(np.mean(fscores)))
print("Confusion Matrix:\n   HC     AD\n{0}".format(np.mean(confusions, axis=0)))

Accuracy: 0.7576381909547739, with std: 0.0181163271125186
Precision: 0.7619164249160348
Recall: 0.7468181818181817
F1 Score: 0.7540476860684628
Confusion Matrix:
   HC     AD
[[ 76.835  23.165]
 [ 25.065  73.935]]
